## Deploy your app with Code Engine
### Introduction
Now that you have Gradio to help you generate a user interface for an application, let's see how you can run applications on IBM Cloud and access it with a public URL using IBM Code Engine.

### Learning objectives
At the end of this project, you will be able to:

* Create a container image
* Create the files required to deploy your app in a container
* Create your Code Engine project
* Build a container image with Code Engine
* Deploy a containerized app using Code Engine

### Container images and containers
Code Engine lets you run your apps in containers on IBM Cloud. A container is an isolated environment or place where an application can run independently. Containers can run anywhere, such as on operating systems, virtual machines, developer's machines, physical servers, and so on. This allows the containerized application to run anywhere as well and the isolation mechanism ensures that the running application will not interfere with the rest of the system.

Containers are created from container images. A container image is basically a snapshot or a blueprint that indicates what will be in a container when it runs. Therefore, to deploy a containerized app, you first need to create the app's container image.

### Creating the container image
The files required to deploy your app in a container are as follows:

* You can use the Gradio framework for generating the user interface of the app,for example, the Python script that contains the code to create and launch the gradio.Interface can be named as `demo.py`.
* The source code of the app has its dependencies, such as libraries that the code uses. Hence, you need a `requirements.txt` file that specifies all the libraries the source code depends on.
* You need a file that shows the container runtime the steps for assembling the container image, named as `Dockerfile`.

Let's create the three files. Open a terminal and while you are in the `home/project` directory, make a new directory `myapp` for storing the files and go into the directory with the following command:

``` 
mkdir myapp
cd myapp
```

Now that you are in the `myapp` directory, run the following command to create the files:

`touch demo.py Dockerfile requirements.txt`


#### Step 1: Creating requirements.txt
If you are a Data Scientist, you may be familiar with the `pip3 install <library-name>` command for installing libraries. By using a `requirements.txt` file that consists of all libraries you need, you can install all of them into your environment at once with the command `pip3 install -r requirements.txt`.

Your goal is to deploy the app in a container; thus, all the dependencies need to go into the container as well.

Let's create a `requirements` file to cover all the libraries and dependencies your app may need.

Open the `requirements.txt` file in `/myapp` and paste the following library names into the file.

`gradio==4.17.0`

##### Testing requirements.txt locally
Since you have already installed the required packages in the "QuickStart Gradio" section, you don't need to install them again here. However, it's important to test the application locally before launching it through Docker to ensure that it runs smoothly without any errors.

You can test to see if the file works correctly by executing the following command in the terminal of your current CloudIDE working environment:

`pip3 install -r requirements.txt`

*Note: Make sure you are in the myapp directory and your virtual environment my_env created previously is activated. This allows the libraries to be installed into your virtual environment only.*

#### Step 2: Creating demo.py
The following code guides you creating a simple Gradio web application. If you want to know more about Gradio, please refer to here.

Open the `demo.py` file in `/myapp` and fill the empty script with the following code.

In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch(server_name="0.0.0.0", server_port= 7860)

Let's test your application and make sure it can run properly.

##### Testing demo.py locally
Open your terminal and go to myapp directory with `cd myapp` command.

If you have successfully executed `pip3 install -r requirements.txt` in the previous step, you are good to go. If not, please run the command now to have the required libraries installed in your working environment.

Run the following command in the terminal:

`python3 demo.py`

If you run this script properly, you should see the following result in your terminal:

The result shows that your app is downloaded and the app is running on http://0.0.0.0:7860/. Click on the button below to access the web app hosted in the CloudIDE:


#### Step 3: Creating Dockerfile
The `Dockerfile` is the blueprint for assembling a container image.

Open the Dockerfile in `/myapp` and paste the following commands into the file.


In [ ]:
FROM python:3.10

WORKDIR /app
COPY requirements.txt requirements.txt
RUN pip3 install --no-cache-dir -r requirements.txt

COPY . .

CMD ["python", "demo.py"]

##### What does the Dockerfile do?
**FROM python:3.10**
Docker images can be inherited from other images. Therefore, instead of creating your own base image, you will use the official Python image `python:3.10` that already has all the tools and packages that you need to run a Python application.

**WORKDIR /app**
To facilitate the running of your commands, let's create a working directory `/app`. This instructs Docker to use this path as the default location for all subsequent commands. By creating the directory, you do not have to type out full file paths but can use relative paths based on the working directory.

**COPY requirements.txt requirements.txt**
Before you run `pip3 install`, you need to get your `requirements.txt` file into your image. You can use the `COPY` command to transfer the contents. The `COPY` command takes two parameters. The first parameter indicates to the Docker what file(s) you would like to copy into the image. The second parameter indicates to the Docker the location where the file(s) need to be copied. You can move the `requirements.txt` file into your working directory /app.

**RUN pip3 install –no-cache-dir -r requirements.txt**
Once you have your `requirements.txt` file inside the image, you can use the `RUN` command to execute the command `pip3 install --no-cache-dir -r requirements.txt`. This works exactly the same as if you were running the command locally on your machine, but this time the modules are installed into the image.

**COPY**
At this point, you have an image that is based on Python version 3.10 and you have installed your dependencies. The next step is to add your source code to the image. You will use the `COPY` command just like you did with your `requirements.txt` file above to copy everything in your current working directory to the file system in the container image.

**CMD ["python", "demo.py"]**
Now, you have to indicate to the Docker what command you want to run when your image is executed inside a container. You use the CMD command. Docker will run the `python demo.py` command to launch your app inside the container.

Now that all three files have been created, let's bring in the Code Engine for building the container image.